In [ ]:
%%capture
!pip install transformers pytorch-lightning --quiet
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [ ]:
import math
import matplotlib.pyplot as plt
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

from huggingface_hub import HfApi, HfFolder, Repository, notebook_login
from google.colab import drive
from pathlib import Path
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchvision.datasets import ImageFolder

In [ ]:
data_dir = Path("images")

if data_dir.exists():
    shutil.rmtree(data_dir)

' for search_term in search_terms:\n    search_term_dir = data_dir / search_term\n    search_term_dir.mkdir(exist_ok=True, parents=True)\n    urls = get_image_urls_by_term(search_term)\n    print(f"Saving images of {search_term} to {str(search_term_dir)}...")\n    urls_to_image_folder(urls, search_term_dir) '

In [ ]:
drive.mount("/content/gdrive")
!unzip /content/gdrive/MyDrive/images.zip -d images

Streaming output truncated to the last 5000 lines.
  inflating: images/chicken_bistek/3AANd9GcSjAsKTFdksRlmZE05ztb6wOUiUtyOAXGJ1sraSkm17SuAa-Xy9.jpg  
  inflating: images/chicken_bistek/3AANd9GcSLpqAD9cZaIuGKYR4LoGBolixU4XdxC22MajHAyhehh_ClEQhx.jpg  
  inflating: images/chicken_bistek/3AANd9GcSMMc8qXvuvI9a1G0iPofsquCIqbULQGoPips3xdF1YZSXGwKAG.jpg  
  inflating: images/chicken_bistek/3AANd9GcSOCh2C8mgUzkBMaruM_iW8MrGT-JOP7I9oDpwQ3QfL-gBGSWxe.jpg  
  inflating: images/chicken_bistek/3AANd9GcSP22sw3xXkEsYhOzDASS5mtq9sA85hUZ1S55W5uyj66neC2cxl.jpg  
  inflating: images/chicken_bistek/3AANd9GcSpKI5SRCxdbwluqjIuhp_JFAquHgSWbLciRrZrHjwQJWSrV16q.jpg  
  inflating: images/chicken_bistek/3AANd9GcSq0-pIOrylPmmU86Y1X2S2zJoSHLL_V3FEmi-YriLtbRBL3yCm.jpg  
  inflating: images/chicken_bistek/3AANd9GcSqAOnkrtbXOnC4AleEGpcyQCSZQ7xTof-XsgY0E8Rh8ZjWTKJn.jpg  
  inflating: images/chicken_bistek/3AANd9GcSQUZeK7awr_AUagkFjo3LhzDdmmDw9bXmcWXLp5yTezk1FGxoo.jpg  
  inflating: images/chicken_bistek/3AANd9GcSr0Lbz

In [ ]:
ds = ImageFolder(data_dir)
indices = torch.randperm(len(ds)).tolist() # numbers from 0 to 6749 which are the indices of each image
n_val = math.floor(len(indices) * .15)
train_ds = torch.utils.data.Subset(ds, indices[:-n_val]) # 5758 images, 85%
val_ds = torch.utils.data.Subset(ds, indices[-n_val:])  # 1012 images, 15%
# 6750 images
# indices, len(ds), n_val, len(indices), len(indices) * .15, train_ds, val_ds
len(indices[:-n_val]), len(indices[-n_val:])

(5738, 1012)

In [ ]:
# assign id to each label
label2id = {}
id2label = {}

for i, class_name in enumerate(ds.classes):
    label2id[class_name] = str(i)
    id2label[str(i)] = class_name

In [ ]:
#
class ImageClassificationCollator:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor

    def __call__(self, batch):
        encodings = self.feature_extractor([x[0] for x in batch], return_tensors="pt")
        encodings["labels"] = torch.tensor([x[1] for x in batch], dtype=torch.long)
        return encodings

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k", from_pt=True)
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label
)
collator = ImageClassificationCollator(feature_extractor)
train_loader = DataLoader(train_ds, batch_size=8, collate_fn=collator, num_workers=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8, collate_fn=collator, num_workers=2)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class Classifier(pl.LightningModule):
    def __init__(self, model, lr: float=2e-5, **kwargs):
        super().__init__()
        self.save_hyperparameters("lr", *list(kwargs))
        self.model = model
        self.forward = self.model.forward
        self.val_acc = Accuracy(
            task="multiclass" if model.config.num_labels > 2 else "binary",
            num_classes=model.config.num_labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log(f"train_loss", outputs.loss)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log(f"val_loss", outputs.loss)
        acc = self.val_acc(outputs.logits.argmax(1), batch["labels"]) # 100 batch size
        self.log(f"val_acc", acc, prog_bar=True)
        return outputs.loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
pl.seed_everything(42)
classifier = Classifier(model, lr=2e-5)
trainer = pl.Trainer(accelerator="gpu", devices=1, precision=16, max_epochs=10)
trainer.fit(classifier, train_loader, val_loader)

INFO:lightning_fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type                      | Params
------------------------------------------------------
0 | model   | ViTForImageClassification | 85.8 M
1 | val_acc | MulticlassAccuracy        | 0     
-

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
val_batch = next(iter(val_loader))
outputs = model(**val_batch)
print('Preds: ', outputs.logits.softmax(1).argmax(1))
print('Labels:', val_batch['labels'])

Preds:  tensor([42, 43, 42, 16, 48, 59, 10, 21])
Labels: tensor([40, 38, 24, 18, 48, 59, 10, 21])


In [ ]:
notebook_login()

In [ ]:
model_id = "try-finding-it"

description = """
Food model
"""
task_name = "Image Classification"
task_type = 'image-classification'
metric_name = 'Accuracy'
metric_type = 'accuracy'
metric_value = trainer.callback_metrics['val_acc'].item()

if Path('./model').exists():
    shutil.rmtree('./model')

token = HfFolder().get_token()
if not token:
    raise RuntimeError("You must log in to push to hub! Run notebook_login() in the cell above.")

hf_username = HfApi().whoami()['name']
model_url = HfApi().create_repo(token=token, repo_id=model_id, exist_ok=True)
model_repo = Repository("./model", clone_from=model_url, use_auth_token=token, git_email=f"{hf_username}@users.noreply.huggingface.co", git_user=hf_username)
model.save_pretrained(model_repo.local_dir)
feature_extractor.save_pretrained(model_repo.local_dir)

tensorboard_logs_path = next(Path(trainer.logger.log_dir).glob('events.out*'))
model_repo_logs_path = Path(model_repo.local_dir) / 'runs'
model_repo_logs_path.mkdir(exist_ok=True, parents=True)
shutil.copy(tensorboard_logs_path, model_repo_logs_path)

example_images_dir = Path(model_repo.local_dir) / 'images'
example_images_dir.mkdir(exist_ok=True, parents=True)
image_markdown_template = '''
#### {class_name}

![{class_name}](images/{example_image_path})
'''
example_images_markdown = ""
for class_idx, class_name in enumerate(ds.classes):
    folder = ds.root / class_name
    image_path = sorted(folder.glob('*'))[0]
    example_image_path = example_images_dir / f"{class_name.replace(' ', '_')}{image_path.suffix}"
    shutil.copy(image_path, example_image_path)
    example_images_markdown += image_markdown_template.format(
        class_name=class_name,
        example_image_path=example_image_path.name
    )

readme_txt = f"""
---
tags:
- image-classification
- pytorch
- huggingpics
metrics:
- {metric_type}

model-index:
- name: {model_id}
  results:
  - task:
      name: {task_name}
      type: {task_type}
    metrics:
      - name: {metric_name}
        type: {metric_type}
        value: {metric_value}
---

# {model_id}

{description}

## Example Images

{example_images_markdown}

""".strip()

(Path(model_repo.local_dir) / 'README.md').write_text(readme_txt)

commit_url = model_repo.push_to_hub()

print("Check out your model at:")
print(f"https://huggingface.co/{hf_username}/{model_id}")

Cloning https://huggingface.co/REDACTED/ into local empty directory.
Adding files tracked by Git LFS: ['images/afritada.jpg', 'images/arroz_caldo.jpg', 'images/bagnet.jpg', 'images/balut.jpg', 'images/bibimbap.jpg', 'images/bibingka.jpg', 'images/bicol_express.jpg', 'images/bistek_tagalog.jpg', 'images/buko_pie.jpg', 'images/bulalo.jpg', 'images/burger.jpg', 'images/calamares.jpg', 'images/cassava_cake.jpg', 'images/champorado.jpg', 'images/chicharon.jpg', 'images/chicken_adobo.jpg', 'images/chicken_bistek.jpg', 'images/chicken_inasal.jpg', 'images/crispy_pata.jpg', 'images/cupcakes.jpg', 'images/donuts.jpg', 'images/filipino_spaghetti.jpg', 'images/french_fries.jpg', 'images/fried_rice.jpg', 'images/ginataang_gulay.jpg', 'images/halo-halo.jpg', 'images/hotdog_sandwich.jpg', 'images/hotsilog.jpg', 'images/ice_cream.jpg', 'images/kaldereta.jpg', 'images/kare-kare.jpg', 'images/kinilaw.jpg', 'images/laing.jpg', 'images/lasagna.jpg', 'images/leche_flan.jpg', 'images/lechon_kawali.jpg', 'i

Upload file model.safetensors:   0%|          | 1.00/327M [00:00<?, ?B/s]

Upload file images/pancakes.jpg:   0%|          | 1.00/20.8k [00:00<?, ?B/s]

Upload file runs/events.out.tfevents.1699622559.7af1d4e5b83d.183.0:   0%|          | 1.00/14.7k [00:00<?, ?B/s…

Upload file images/cupcakes.jpg:   0%|          | 1.00/21.5k [00:00<?, ?B/s]

...

Upload file images/buko_pie.jpg:   0%|          | 1.00/5.50k [00:00<?, ?B/s]

Upload file images/pork_barbecue.jpg:   0%|          | 1.00/5.49k [00:00<?, ?B/s]

Upload file images/balut.jpg:   0%|          | 1.00/5.46k [00:00<?, ?B/s]

To https://huggingface.co/REDACTED/
   6f6eed7..1980c50  main -> main

   6f6eed7..1980c50  main -> main



Check out your model at:
https://huggingface.co/REDACTED/
